
#### 1. Objetivo Final do Projeto

O objetivo é criar um modelo de Inteligência Artificial especializado em **resumir notícias**. Para isso, vamos usar uma técnica chamada **Fine-Tuning**.

#### 2. O que é Fine-Tuning?

Fine-tuning (ou ajuste fino) é o processo de pegar um modelo de linguagem pré-treinado (como o GPT da OpenAI) e continuar seu treinamento com um conjunto de dados específico para uma tarefa particular.

*   **Modelo Base:** É um modelo gigante, como o GPT-3.5, que já foi treinado com uma quantidade massiva de texto da internet e "sabe" sobre linguagem, fatos, raciocínio, etc. Pense nele como um recém-formado com conhecimento geral.
*   **Ajuste Fino:** Nós o especializamos em uma tarefa (como resumir notícias) mostrando a ele muitos exemplos de como queremos que ele execute essa tarefa. É como dar uma pós-graduação ao recém-formado.

O resultado é um modelo menor, mais rápido e mais consistente para a sua tarefa específica do que o modelo base genérico.

#### 3. O Pipeline de Preparação de Dados (O Foco da Aula)

Para fazer o fine-tuning, precisamos de um conjunto de dados de alta qualidade no formato `(entrada, saída esperada)`. No nosso caso, seria `(texto da notícia, resumo da notícia)`. A aula demonstra como criar esse dataset do zero.

---



**Etapa 1: Coleta de Dados Brutos (Web Scraping)**

Como não temos um dataset pronto, vamos buscá-lo na internet. O processo de extrair informações de sites de forma automatizada é chamado de **Web Scraping**.

*   **Ferramentas Utilizadas:**
    *   `requests`: Uma biblioteca Python para fazer requisições HTTP, ou seja, para "baixar" o código HTML de uma página da web.
    *   `BeautifulSoup`: Uma biblioteca para "navegar" e extrair informações de um documento HTML. Ela entende a estrutura de tags (como `<div>`, `<p>`, `<a>`) e nos permite encontrar o que queremos.

*   **Processo em Duas Fases:**
    1.  **`news-scrapper.ipynb` (Coletor de Links):** O primeiro script visita uma página principal de um portal de notícias (no caso, a seção "World" da CNN). Ele usa `BeautifulSoup` para encontrar todas as tags `<a>` (que representam links) e filtra para pegar apenas os links que levam a artigos de notícias. Esses links (URLs) são salvos em um arquivo de texto (`CNN_Links.txt`).
    2.  **`get-news-content.ipynb` (Extrator de Conteúdo):** O segundo script lê o arquivo `CNN_Links.txt`. Para cada link na lista, ele visita a página do artigo, usa `BeautifulSoup` novamente para encontrar o elemento HTML que contém o texto principal da notícia (geralmente um `<div>` com uma classe específica como `article__content`) e extrai todo o texto. O resultado é salvo em um arquivo JSON (`news_contents.json`) contendo uma lista de todos os textos das notícias coletadas.

---

In [ ]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO E CONFIGURAÇÃO INICIAL
# ==============================================================================
# Esta célula combina as configurações iniciais de todos os notebooks.

# Instala as bibliotecas necessárias que estavam faltando nos notebooks originais.
!pip install requests beautifulsoup4 openai

# Importa as bibliotecas que serão usadas ao longo do projeto.
import requests
from bs4 import BeautifulSoup
import json
import os
from openai import OpenAI
from google.colab import drive
from google.colab import userdata # Import userdata to access Colab secrets

# Monta o Google Drive para salvar nossos arquivos de forma persistente.
# Isso corrige o problema de salvar arquivos em um ambiente temporário.
drive.mount('/content/drive')

# --- IMPORTANTE: SUBSTITUA PELA SUA CHAVE DE API DA OPENAI ---
# Acesse a chave de API do OpenAI armazenada no Colab Secrets Manager
# Certifique-se de ter adicionado sua chave lá com o nome 'OPENAI_API_KEY'
OPENAI_API_KEY = None
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if OPENAI_API_KEY is None:
        raise ValueError("OPENAI_API_KEY not found in Colab Secrets Manager.")
except Exception as e:
    print(f"Erro ao obter a chave de API: {e}")
    print("Por favor, adicione sua chave de API da OpenAI ao Colab Secrets Manager com o nome 'OPENAI_API_KEY'.")
    #exit() # You might want to exit if the key is not found

client = None
if OPENAI_API_KEY:
  client = OpenAI(api_key=OPENAI_API_KEY)
  print("OpenAI client initialized successfully.")
else:
  print("OpenAI client not initialized due to missing API key.")


# Define um caminho base no Google Drive para organizar todos os arquivos gerados.
# Crie esta estrutura de pastas no seu Drive: MyDrive/FIAP/Fine-tuning/Aula 1
base_path = "/content/drive/MyDrive/FIAP/Fine-tuning/Aula1/"
os.makedirs(base_path, exist_ok=True) # Garante que o diretório exista

print(f"Configuração concluída. Os arquivos serão salvos em: {base_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
OpenAI client initialized successfully.
Configuração concluída. Os arquivos serão salvos em: /content/drive/MyDrive/FIAP/Fine-tuning/Aula1/




**Etapa 2: Geração dos Resumos (Outputs)**

Agora temos as "entradas" (os textos das notícias), mas ainda precisamos das "saídas esperadas" (os resumos). Escrever resumos para centenas de notícias manualmente seria inviável.

*   **Solução (Usando uma IA para criar dados para outra IA):**
    *   **`generate-output-for-news.ipynb`:** Este script utiliza a **API da OpenAI**. Ele pega cada notícia do arquivo `news_contents.json`, envia para um modelo poderoso (como o `gpt-3.5-turbo`) com uma instrução clara ("*Summarize this news article...*") e recebe de volta um resumo gerado pela IA.
    *   O script então junta o texto original da notícia (`story`) com o resumo gerado (`summary`) em um novo arquivo JSON (`news_summaries.json`).

---



In [19]:
# ==============================================================================
# CÉLULA 2: ETAPA 1 - WEB SCRAPING DOS LINKS (Análogo a news-scrapper.ipynb)
# --- VERSÃO FINAL COM FILTROS AVANÇADOS DE VÍDEO E DATA ---
# ==============================================================================
# Esta versão exclui links de vídeo e coleta apenas notícias do dia anterior.

# Adiciona as importações necessárias para manipulação de datas
from datetime import datetime, timedelta

def scrape_cnn_links(url):
    """
    Função que faz o scraping da CNN e retorna uma lista de links de notícias,
    aplicando dois filtros:
    1. Exclui qualquer URL que contenha '/video/'.
    2. Inclui apenas URLs cuja data seja do dia anterior ao da execução.
    """
    print(f"Iniciando scraping da URL: {url}")

    # --- NOVO: LÓGICA DE DATA ---
    # Calcula a data de ontem
    yesterday = datetime.now() - timedelta(days=1)
    print(f"==> Filtro Ativado: Buscando links apenas para a data de ontem: {yesterday.strftime('%Y-%m-%d')}")

    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        valid_links = []

        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']

            # --- NOVO: FILTRO 1 - EXCLUIR VÍDEOS ---
            # Se o link contiver '/video/', pula para a próxima iteração do loop.
            if "/video/" in href:
                continue

            # --- LÓGICA DE ESTRUTURA DE URL (JÁ EXISTENTE) ---
            path_parts = href.strip('/').split('/')

            if len(path_parts) >= 3 and path_parts[0].isdigit() and len(path_parts[0]) == 4:

                # --- NOVO: FILTRO 2 - VERIFICAR A DATA ---
                try:
                    url_year = int(path_parts[0])
                    url_month = int(path_parts[1])
                    url_day = int(path_parts[2])

                    # Compara a data do link com a data de ontem
                    if (url_year == yesterday.year and
                        url_month == yesterday.month and
                        url_day == yesterday.day):

                        # Se todos os filtros passaram, o link é válido
                        full_link = f"https://edition.cnn.com{href}"
                        valid_links.append(full_link)

                except (ValueError, IndexError):
                    # Ignora links que parecem ter uma data mas têm um formato inválido
                    continue

        unique_links = sorted(list(set(valid_links)))
        print(f"Encontrados {len(unique_links)} links de notícias únicos para a data de ontem.")
        return unique_links

    except requests.exceptions.RequestException as e:
        print(f"Ocorreu um erro de rede ou HTTP: {e}")
        return []
    except Exception as e:
        print(f"Um erro inesperado ocorreu: {e}")
        return []

# URL da seção "World" da CNN, como no notebook
cnn_url = "https://edition.cnn.com/world"
links_list = scrape_cnn_links(cnn_url)

# Salva os links no caminho definido no Google Drive
links_file_path = os.path.join(base_path, "CNN_Links.txt")

if links_list:
    with open(links_file_path, 'w') as file:
        for link in links_list:
            file.write(link + '\n')
    print(f"Links salvos com sucesso em: {links_file_path}")
else:
    print("Nenhum link correspondente aos filtros foi encontrado. O arquivo de links não foi atualizado.")

Iniciando scraping da URL: https://edition.cnn.com/world
==> Filtro Ativado: Buscando links apenas para a data de ontem: 2025-11-03
Encontrados 14 links de notícias únicos para a data de ontem.
Links salvos com sucesso em: /content/drive/MyDrive/FIAP/Fine-tuning/Aula1/CNN_Links.txt



**Etapa 3: Formatação Final do Dataset**

Os modelos de fine-tuning exigem que os dados estejam em um formato específico. Normalmente, é um arquivo **JSON Lines (`.jsonl`)**, onde cada linha é um objeto JSON independente.

*   **`prepare-data.ipynb`:** O último script faz essa formatação.
    1.  Ele lê os dados que geramos (`news_summaries.json`) e também um **dataset complementar** (um arquivo `data.jsonl` já pronto, para aumentar o volume de dados).
    2.  Para cada par `(story, summary)`, ele cria uma string formatada que serve como um único exemplo de treinamento. O formato é crucial e geralmente segue um padrão como:
        ```
        "input": "### SUMMARIZE THIS NEWS.\n\n[Texto da notícia]\n\n###[Resumo da notícia]###"
        ```
    3.  Essa estrutura ensina o modelo: "Quando você vir essa instrução e esse texto, a sua resposta deve ser este resumo."
    4.  Todos esses exemplos formatados são salvos em um único arquivo final (`news_dataset_chat_data.json`), que está pronto para ser usado no processo de fine-tuning.



In [20]:
# ==============================================================================
# CÉLULA 3: ETAPA 2 - EXTRAÇÃO DO CONTEÚDO (Análogo a get-news-content.ipynb)
# --- VERSÃO DEFINITIVA COM EXTRAÇÃO DE JSON-LD (MÉTODO PREFERENCIAL) ---
# ==============================================================================
# Este scraper prioriza a extração de dados estruturados (JSON-LD) e usa o
# scraping de HTML visual como um fallback, tornando-o extremamente robusto.

def get_news_content(links_file):
    """
    Lê o arquivo de links e extrai o conteúdo de cada artigo.
    Primeiro, tenta extrair o texto limpo do JSON-LD estruturado ('articleBody').
    Se falhar, recorre ao scraping do HTML visual como fallback.
    """
    print("\nIniciando extração de conteúdo com scraper definitivo (JSON-LD + Fallback)...")
    try:
        with open(links_file, 'r') as file:
            links = file.readlines()
    except FileNotFoundError:
        print(f"Erro: O arquivo de links '{links_file}' não foi encontrado. Execute a Célula 2 primeiro.")
        return None

    news_contents = []

    for i, link in enumerate(links):
        link = link.strip()
        if not link:
            continue

        print(f"Processando link {i+1}/{len(links)}: {link}")
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(link, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            content = None # Variável para armazenar o conteúdo encontrado

            # --- TENTATIVA 1: MÉTODO PREFERENCIAL (JSON-LD) ---
            json_ld_scripts = soup.find_all('script', type='application/ld+json')
            for script in json_ld_scripts:
                try:
                    # O conteúdo do script é uma string, então precisamos carregá-lo como JSON
                    data = json.loads(script.string)
                    # O JSON pode ser uma lista ou um dicionário, então lidamos com ambos os casos
                    if isinstance(data, list):
                        for item in data:
                            if isinstance(item, dict) and 'articleBody' in item:
                                content = item['articleBody']
                                print("  -> Sucesso: Conteúdo extraído via JSON-LD.")
                                break # Encontrou, sai do loop interno
                    elif isinstance(data, dict) and 'articleBody' in data:
                        content = data['articleBody']
                        print("  -> Sucesso: Conteúdo extraído via JSON-LD.")
                    if content:
                        break # Encontrou, sai do loop de scripts
                except (json.JSONDecodeError, TypeError):
                    # Ignora scripts malformados ou vazios e continua para o próximo
                    continue

            # --- TENTATIVA 2: MÉTODO FALLBACK (SCRAPING DE HTML) ---
            if not content:
                print("  -> Aviso: JSON-LD não encontrado ou sem 'articleBody'. Tentando fallback de HTML...")
                main_container = soup.find('div', class_='article__content-container')
                if not main_container:
                    main_container = soup.find('div', class_='article__content')

                if main_container:
                    paragraphs_elements = main_container.find_all('div', {'data-component-name': 'paragraph'})
                    if not paragraphs_elements:
                        paragraphs_elements = main_container.find_all('p', class_='paragraph')

                    if paragraphs_elements:
                        paragraphs_text = [p.get_text(strip=True) for p in paragraphs_elements]
                        content = ' '.join(paragraphs_text)
                        print("  -> Sucesso: Conteúdo extraído via Fallback de HTML.")

            # Adiciona o resultado final à lista
            if content:
                news_contents.append(content)
            else:
                news_contents.append("Conteúdo não encontrado (ambos os métodos falharam).")
                print("  -> Falha: Nenhum método conseguiu extrair o conteúdo.")

        except requests.exceptions.RequestException as e:
            print(f"  -> Erro de rede ao acessar o link: {e}")
            news_contents.append(f"Erro de rede: {e}")
        except Exception as e:
            print(f"  -> Erro inesperado ao processar o link: {e}")
            news_contents.append(f"Erro inesperado: {e}")

    return {"news_content": news_contents}

# Chama a função para extrair os conteúdos
contents_data = get_news_content(links_file_path)

# Apenas continua se a extração foi bem-sucedida
if contents_data:
    # Salva o conteúdo em um arquivo JSON no Google Drive
    contents_file_path = os.path.join(base_path, "news_contents.json")
    with open(contents_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(contents_data, json_file, indent=4)
    print(f"\nConteúdos das notícias salvos com sucesso em: {contents_file_path}")


Iniciando extração de conteúdo com scraper definitivo (JSON-LD + Fallback)...
Processando link 1/14: https://edition.cnn.com/2025/11/03/africa/trump-christian-killings-nigeria-intl
  -> Sucesso: Conteúdo extraído via JSON-LD.
Processando link 2/14: https://edition.cnn.com/2025/11/03/americas/day-of-the-dead-mexico-traditions-intl-latam
  -> Sucesso: Conteúdo extraído via JSON-LD.
Processando link 3/14: https://edition.cnn.com/2025/11/03/americas/jamaica-black-river-destruction-after-hurricane-melissa-intl-latam
  -> Sucesso: Conteúdo extraído via JSON-LD.
Processando link 4/14: https://edition.cnn.com/2025/11/03/asia/himalaya-peak-avalanche-nepal-intl
  -> Sucesso: Conteúdo extraído via JSON-LD.
Processando link 5/14: https://edition.cnn.com/2025/11/03/asia/japan-takaichi-north-korea-kim-jong-un-intl-hnk
  -> Sucesso: Conteúdo extraído via JSON-LD.
Processando link 6/14: https://edition.cnn.com/2025/11/03/asia/north-korea-kim-yong-nam-death-intl-hnk
  -> Sucesso: Conteúdo extraído via

In [24]:
# ==============================================================================
# CÉLULA 4: ETAPA 3 - GERAÇÃO DOS RESUMOS (Análogo a generate-output-for-news.ipynb)
# --- VERSÃO CORRIGIDA COM PARSER DE JSON ROBUSTO ---
# ==============================================================================
# Esta versão lida com respostas malformadas da API da OpenAI, extraindo o JSON
# válido mesmo que ele esteja cercado por texto adicional.

def summarize_news(news_file):
    """
    Carrega o conteúdo das notícias e gera resumos para cada uma usando a API da OpenAI,
    com um parser de resposta robusto para evitar erros de JSON.
    """
    print("\nIniciando geração de resumos com a API da OpenAI (parser robusto)...")
    try:
        with open(news_file, 'r', encoding='utf-8') as file:
            news_data = json.load(file)
    except FileNotFoundError:
        print(f"Erro: O arquivo '{news_file}' não foi encontrado. Execute a Célula 3 primeiro.")
        return None

    news_contents = news_data.get('news_content', [])
    summaries = []

    for i, content in enumerate(news_contents):
        print(f"Gerando resumo para a notícia {i+1}/{len(news_contents)}...")

        # Filtro para não enviar conteúdo inválido ou muito curto para a API
        if not content or len(content) < 150 or "Conteúdo não encontrado" in content:
            summary_obj = {"story": content, "summary": "Conteúdo insuficiente para resumir."}
            summaries.append(summary_obj)
            continue

        try:
            response = client.chat.completions.create(
              model="gpt-3.5-turbo",
              response_format={ "type": "json_object" },
              messages=[
                {"role": "system", "content": 'You are a JSON summarization bot. Your only task is to summarize the provided news article. You MUST respond with ONLY a valid JSON object in the format {"summary": "your_summary_here"}. Do not include any other text, markdown, or explanations.'},
                {"role": "user", "content": f"{content}"}
              ],
              temperature=0.5,
              max_tokens=200
            )

            raw_response_string = response.choices[0].message.content
            summary_text = "Erro ao extrair o resumo do JSON."

            # --- NOVO: LÓGICA DE PARSING ROBUSTA ---
            try:
                # Tentativa 1: Parse direto
                summary_data = json.loads(raw_response_string)
                summary_text = summary_data.get("summary", summary_text)
            except json.JSONDecodeError:
                print("  -> Aviso: Resposta da OpenAI não é um JSON válido. Tentando extrair...")
                # Tentativa 2: Extrair o JSON do meio da string
                try:
                    start_index = raw_response_string.find('{')
                    end_index = raw_response_string.rfind('}') + 1
                    if start_index != -1 and end_index != 0:
                        clean_json_string = raw_response_string[start_index:end_index]
                        summary_data = json.loads(clean_json_string)
                        summary_text = summary_data.get("summary", summary_text)
                        print("  -> Sucesso: JSON extraído com sucesso.")
                    else:
                         print("  -> Falha: Não foi possível encontrar um objeto JSON na resposta.")
                except json.JSONDecodeError as e:
                    print(f"  -> Falha: Erro ao decodificar o JSON extraído. Erro: {e}")
                    print(f"  -> Resposta Bruta com Problema: {raw_response_string}")

            summary_obj = {"story": content, "summary": summary_text}
            summaries.append(summary_obj)

        except Exception as e:
            print(f"  -> Erro na API da OpenAI: {e}")
            summaries.append({"story": content, "summary": "Erro na chamada da API."})

    return {"news_summaries": summaries}

# Chama a função para gerar os resumos
summaries_data = summarize_news(contents_file_path)

# Apenas continua se a geração foi bem-sucedida
if summaries_data:
    # Salva os resultados (notícia + resumo) no Google Drive
    summaries_file_path = os.path.join(base_path, "news_summaries.json")
    with open(summaries_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(summaries_data, json_file, indent=4)
    print(f"\nResumos gerados e salvos em: {summaries_file_path}")


Iniciando geração de resumos com a API da OpenAI (parser robusto)...
Gerando resumo para a notícia 1/14...
Gerando resumo para a notícia 2/14...
Gerando resumo para a notícia 3/14...
Gerando resumo para a notícia 4/14...
Gerando resumo para a notícia 5/14...
Gerando resumo para a notícia 6/14...
Gerando resumo para a notícia 7/14...
Gerando resumo para a notícia 8/14...
Gerando resumo para a notícia 9/14...
Gerando resumo para a notícia 10/14...
Gerando resumo para a notícia 11/14...
Gerando resumo para a notícia 12/14...
Gerando resumo para a notícia 13/14...
Gerando resumo para a notícia 14/14...

Resumos gerados e salvos em: /content/drive/MyDrive/FIAP/Fine-tuning/Aula1/news_summaries.json


In [25]:


# ==============================================================================
# CÉLULA 5: ETAPA 4 - FORMATAÇÃO FINAL DO DATASET (Análogo a prepare-data.ipynb)
# ==============================================================================
# Fiel à lógica e ao formato exato de prompt do quarto notebook.

def format_final_dataset(summaries_file):
    """
    Lê o arquivo com notícias e resumos e formata no padrão final para o fine-tuning.
    """
    print("\nFormatando dados para o fine-tuning...")
    processed_data = []

    with open(summaries_file, 'r', encoding='utf-8') as file:
        json_data = json.load(file)

    news_list = json_data.get("news_summaries", [])

    for item in news_list:
        story = item.get("story", "")
        summary = item.get("summary", "")

        # Ignora entradas inválidas ou com erro
        if not story or not summary or "erro" in summary.lower() or "insuficiente" in summary.lower():
            continue

        # O formato do texto é exatamente o mesmo que o professor demonstrou
        formatted_text = f"SUMMARIZE THIS NEWS.\n[|News|] {story}[|eNews|]\n\n[|summary|]{summary}[|esummary|]"
        processed_data.append({"input": formatted_text})

    return processed_data

# Processa o arquivo de resumos que criamos
final_data = format_final_dataset(summaries_file_path)

# Salva o dataset final no formato JSON, como no notebook do professor
output_filename = os.path.join(base_path, "news_dataset_chat_data.json")
with open(output_filename, 'w', encoding='utf-8') as file:
    # O professor salva como um grande arquivo JSON, não JSONL. Vamos manter essa fidelidade.
    json.dump(final_data, file, ensure_ascii=False, indent=4)

print(f"\nPIPELINE CONCLUÍDO!")
print(f"Dataset final para fine-tuning foi gerado com sucesso.")
print(f"Total de exemplos formatados: {len(final_data)}")
print(f"Arquivo salvo em: {output_filename}")


Formatando dados para o fine-tuning...

PIPELINE CONCLUÍDO!
Dataset final para fine-tuning foi gerado com sucesso.
Total de exemplos formatados: 13
Arquivo salvo em: /content/drive/MyDrive/FIAP/Fine-tuning/Aula1/news_dataset_chat_data.json


---

### Programa Análogo Completo (Google Colab)

Aqui está um único notebook do Google Colab que combina e executa todas as etapas mostradas na aula. Para executá-lo, basta copiar e colar o código em um novo notebook no [Google Colab](https://colab.research.google.com/).

**Instruções:**
1.  **Obtenha uma Chave de API da OpenAI:** Você precisará criar uma conta em [platform.openai.com](https://platform.openai.com/) e gerar uma chave de API na seção "API keys".
2.  **Substitua a Chave:** No código abaixo, substitua o texto `"your_openai_api_key"` pela sua chave real.
3.  **Execute as Células:** Execute cada célula de código em ordem.
